In [1]:
import nbaPlayerStatModeling as nbaModel

In [2]:
nba = nbaModel.playerStatModel(
        day_offset = 0, 
        season = '2024-25', 
        perMode = 'PerGame',
        num_simulations = 200000
)

In [3]:
nba.get_teams_playing()
nba.get_teams_data(
    base_url = 'https://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=0&LeagueID=00&Location=&MeasureType={stattype}&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={sid}&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision='
)
nba.get_players_playing(
    url_base_nba_player_stat = 'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode={per}&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={sid}&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID={tid}&TwoWay=0&VsConference=&VsDivision=&Weight=',
    minute_cutoff = 0
)
nba.get_props()


5  games today...
179 players returned from 10 teams, out of a total 10
681 prop bets for 70 players...


In [4]:
#nba.df_players = nba.df_players.iloc[1:10, :]
nba.get_player_home_adv(
    use_default=False,
    sleep_time = 2
)

1 / 70 player splits..
2 / 70 player splits..
3 / 70 player splits..
4 / 70 player splits..
5 / 70 player splits..
6 / 70 player splits..
7 / 70 player splits..
8 / 70 player splits..
9 / 70 player splits..
10 / 70 player splits..
11 / 70 player splits..
12 / 70 player splits..
13 / 70 player splits..
14 / 70 player splits..
15 / 70 player splits..
16 / 70 player splits..
17 / 70 player splits..
18 / 70 player splits..
19 / 70 player splits..
20 / 70 player splits..
21 / 70 player splits..
22 / 70 player splits..
23 / 70 player splits..
24 / 70 player splits..
25 / 70 player splits..
26 / 70 player splits..
27 / 70 player splits..
28 / 70 player splits..
29 / 70 player splits..
30 / 70 player splits..
31 / 70 player splits..
32 / 70 player splits..
33 / 70 player splits..
34 / 70 player splits..
35 / 70 player splits..
36 / 70 player splits..
37 / 70 player splits..
38 / 70 player splits..
39 / 70 player splits..
40 / 70 player splits..
41 / 70 player splits..
42 / 70 player splits..
4

In [5]:
df_copy = nba.df_players.copy()

In [51]:
nba.df_players = df_copy.copy()
nba.model_expected_minutes()
nba.calculate_model_inputs()
nba.model_stats()

In [52]:
player = 'Quentin Grimes'
nba.df_players[nba.df_players['PLAYER_NAME']==player][['PLAYER_NAME', 'REB', 'expReb', 'reb_line', 'reb_oOdds', 'REBoOdds','REBoOdds_deci','REBoProb']]

,PLAYER_NAME,REB,expReb,reb_line,reb_oOdds,REBoOdds,REBoOdds_deci,REBoProb
42,Quentin Grimes,6.4,6.997714,4.5,-130.0,-475,1.21,0.826235


In [53]:
nba.df_players[nba.df_players['PLAYER_NAME']==player][['PLAYER_NAME', 'AST', 'expAst', 'ast_line', 'ast_oOdds', 'ASToOdds','ASToOdds_deci','ASToProb']]

,PLAYER_NAME,AST,expAst,ast_line,ast_oOdds,ASToOdds,ASToOdds_deci,ASToProb
42,Quentin Grimes,3.6,3.964722,2.5,-175.0,-312.0,1.32,0.75757


In [54]:
nba.df_players[nba.df_players['PLAYER_NAME']==player][['PLAYER_NAME', 'PTS', 'expPts', 'pts_line', 'pts_oOdds', 'PTSoOdds','PTSoOdds_deci','PTSoProb']]

,PLAYER_NAME,PTS,expPts,pts_line,pts_oOdds,PTSoOdds,PTSoOdds_deci,PTSoProb
42,Quentin Grimes,15.8,14.318904,12.5,-122.0,-155,1.645,0.607895


In [55]:
nba.df_players[nba.df_players['PLAYER_NAME']==player][['PLAYER_NAME', 'FG3M', 'expFG3M', 'threes_line', 'threes_oOdds', 'FG3MoOdds','FG3MoOdds_deci','FG3MoProb']]

,PLAYER_NAME,FG3M,expFG3M,threes_line,threes_oOdds,FG3MoOdds,FG3MoOdds_deci,FG3MoProb
42,Quentin Grimes,2.0,1.577405,1.5,-175.0,-116.0,1.862,0.537045


In [56]:
nba.df_players[nba.df_players['PLAYER_NAME']==player][['PLAYER_NAME', 'PRA', 'expPra', 'pra_line', 'pra_oOdds', 'PRAoOdds','PRAoOdds_deci','PRAoProb']]

,PLAYER_NAME,PRA,expPra,pra_line,pra_oOdds,PRAoOdds,PRAoOdds_deci,PRAoProb
42,Quentin Grimes,25.8,25.281339,20.5,-115.0,-124.0,1.801,0.555375


In [57]:
nba.df_players[nba.df_players['PLAYER_NAME']==player][['PLAYER_NAME', 'PR', 'expPr', 'pr_line', 'pr_oOdds', 'PRoOdds','PRoOdds_deci','PRoProb']]

,PLAYER_NAME,PR,expPr,pr_line,pr_oOdds,PRoOdds,PRoOdds_deci,PRoProb
42,Quentin Grimes,22.2,21.316617,17.5,-120.0,-284,1.351,0.740155


In [58]:
nba.df_players[nba.df_players['PLAYER_NAME']==player][['PLAYER_NAME', 'PA', 'expPa', 'pa_line', 'pa_oOdds', 'PAoOdds','PAoOdds_deci','PAoProb']]

,PLAYER_NAME,PA,expPa,pa_line,pa_oOdds,PAoOdds,PAoOdds_deci,PAoProb
42,Quentin Grimes,19.4,18.283625,15.5,-115.0,158.0,2.585,0.38681


In [59]:
nba.df_players[nba.df_players['PLAYER_NAME']==player][['PLAYER_NAME', 'RA', 'expRa', 'ra_line', 'ra_oOdds', 'RAoOdds','RAoOdds_deci','RAoProb']]

,PLAYER_NAME,RA,expRa,ra_line,ra_oOdds,RAoOdds,RAoOdds_deci,RAoProb
42,Quentin Grimes,10.0,10.962435,7.5,-140.0,111.0,2.115,0.472865


In [60]:
nba.df_players[nba.df_players['PLAYER_NAME']==player][['PLAYER_NAME', 'STL', 'expStl', 'stl_line', 'stl_oOdds', 'STLoOdds','STLoOdds_deci','STLoProb']]

,PLAYER_NAME,STL,expStl,stl_line,stl_oOdds,STLoOdds,STLoOdds_deci,STLoProb
42,Quentin Grimes,0.8,0.643613,0.5,-225.0,110.0,2.104,0.475245


In [62]:
nba.df_players[nba.df_players['PLAYER_NAME']==player][['PLAYER_NAME', 'BLK', 'expBlk', 'blk_line', 'blk_oOdds', 'BLKoOdds','BLKoOdds_deci','BLKoProb']]

,PLAYER_NAME,BLK,expBlk,blk_line,blk_oOdds,BLKoOdds,BLKoOdds_deci,BLKoProb
42,Quentin Grimes,0.6,0.482709,0.5,160.0,160.0,2.608,0.383495


In [63]:
nba.df_players[nba.df_players['PLAYER_NAME']==player][['PLAYER_NAME', 'SB', 'expSb', 'sb_line', 'sb_oOdds', 'SBoOdds','SBoOdds_deci','SBoProb']]

,PLAYER_NAME,SB,expSb,sb_line,sb_oOdds,SBoOdds,SBoOdds_deci,SBoProb
42,Quentin Grimes,1.4,1.126322,0.0,0.0,NaN,NaN,1.0


In [ ]:
#nba.df_players.to_excel('test.xlsx')

In [33]:
pts_cols = []
for i in nba.df_players.columns:
    if  'line' in i.lower():
        pts_cols.append(i)

#nba.df_players[pts_cols]
pts_cols

['threes_line',
 'stl_line',
 'reb_line',
 'ra_line',
 'pts_line',
 'pra_line',
 'pr_line',
 'pa_line',
 'blk_line',
 'ast_line',
 'sb_line']

In [ ]:
nba.df_players.iloc[0:5,6:25]


,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS
0,9.8,0.491,1.4,4,0.332,1.4,1.7,0.792,0.6,2.9,3.5,4.6,1.5,0.9,0.4,0.8,2.3,1.4,12.3
1,14.2,0.427,3.0,8,0.370,2.9,3.3,0.894,0.3,2.9,3.3,4.6,2.4,0.9,0.2,0.9,2.1,3.0,18.0
2,10.4,0.452,1.4,3,0.355,1.7,2.2,0.757,1.7,5.6,7.3,6.3,2.7,1.1,0.6,0.9,1.7,2.0,12.4
3,3.4,0.451,0.6,1,0.337,0.7,0.9,0.745,0.9,1.1,2.0,0.4,0.4,0.4,0.2,0.1,1.3,0.7,4.4
4,7.8,0.226,1.0,4,0.222,0.5,1.0,0.500,1.0,2.5,3.5,2.0,1.3,1.0,0.0,0.5,2.3,1.0,5.0
